In [1]:
#correcting error of not using percent (used fraction) of missing genotypes 
library('data.table')
library('tidyverse')
setwd('~/dpetrov/MarkB/Orchard2021Data/RData/Downsampled/')


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()   masks data.table::between()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::first()     masks data.table::first()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::last()      masks data.table::last()
✖ purrr::transpose() masks data.table::transpose()


In [8]:
##check percent missing 
d.miss = fread('../../founders/vcfs/plink2.vmiss')

In [11]:
colnames(d.miss) = c('chrom', 'id', 'missing_ct', 'obs_ct', 'frac.miss')
d.miss %>% group_by(chrom) %>% summarise(mean.frac.miss = mean(frac.miss))

chrom,mean.frac.miss
<chr>,<dbl>
2L,0.04279916
2R,0.03811139
3L,0.04430592
3R,0.03698547
X,0.06444708


In [48]:
chroms = c("2L", "2R", "3L", "3R", "X")

n.snps = c(nrow(filter(sites, chrom == "2L")), nrow(filter(sites, chrom == "2R")), nrow(filter(sites, chrom == "3L")), nrow(filter(sites, chrom == "3R")), nrow(filter(sites, chrom == "X")))

chrom_length <- 23000000
recomb_rate <- 0.0000000239


#EEC function from S. Greenblum
calc_expected_ec=function(rd,gen,pct_missing,nof_snps,chrom_length,recomb_rate){
  
#example input: rd=5,gen=20,pct_missing=2,nof_snps=283438
# rd: actual read depth
# gen: nof generations since population founding
# pct_missing: percent of founder genotype calls that are missing (%, not a fraction)
# nof_snps: average number of snps per chromosome
# chrom_length: average length of a chromosome
# recomb_rate: average recombination rate

q=18

mycoeffs=data.frame(a=0.5199118,b=-0.6909052,c=0.3553630)

    
winSize=round(qexp(q/100,1/((chrom_length)/((recomb_rate)*(chrom_length)*(gen)+1)))/1000)

    
nReadsPerWin=(rd)*(nof_snps)*(winSize)*1000/(chrom_length)

    
ec = 10^(mycoeffs$a * log10( nReadsPerWin ) + mycoeffs$b * log10(1+pct_missing) + mycoeffs$c )

ec

}


In [52]:
##These are actually likely the fraction missing - not percent
perc.missing = fread('../../founders//inbredv2.filtered.Orch21.PctMissing.csv')
perc.missing

V1,chrom,pct
<int>,<chr>,<dbl>
1,2L,0.04279916
2,2R,0.03811139
3,3L,0.04430592
4,3R,0.03698547
5,X,0.06444708


In [3]:
#how elevated is eec if use fraction instead of percent missing? -- 3.334 x greater
(calc_expected_ec(rd = 8, gen = 14, pct_missing = 0.05, nof_snps = 300000,
                 chrom_length = chrom_length, recomb_rate = recomb_rate))/(calc_expected_ec(rd = 8, gen = 14, pct_missing = 5, nof_snps = 300000,
                 chrom_length = chrom_length, recomb_rate = recomb_rate))

[1] 3.334179

In [4]:
#how elevated is eec if percent missing is 100x greater? - adjusting rd, gen, and n_of snps to see if this changes things
(calc_expected_ec(rd = 6, gen = 5, pct_missing = 0.05, nof_snps = 300000,
                 chrom_length = chrom_length, recomb_rate = recomb_rate))/(calc_expected_ec(rd = 6, gen = 5, pct_missing = 5, nof_snps = 300000,
                 chrom_length = chrom_length, recomb_rate = recomb_rate))

##NO CHANGE - the offset for having the perc. founder missing not be in percent form is ~3.33

[1] 3.334179

In [8]:
##Adjusting eec values for RData - reduce all by 1/3.33
load('./orch2021_Downsampled_META_Filtered.RData')

In [13]:
n.2L = nrow(sites %>% filter(chrom == '2L'))
n.2R = nrow(sites %>% filter(chrom == '2R'))
n.3L = nrow(sites %>% filter(chrom == '3L'))
n.3R = nrow(sites %>% filter(chrom == '3R'))
n.X = nrow(sites %>% filter(chrom == 'X'))

df = cbind(sites, eec)
df.chr = df %>% group_by(chrom) %>% sample_n(1)
df.chr = df.chr[,-c(1:2)]
df.chr = sapply(df.chr,"*",(1/3.33))
df.chr = as.data.frame(df.chr)
df.chr = df.chr %>% slice(rep(1:n(), times = c(n.2L, n.2R, n.3L, n.3R, n.X)))
df.chr = as.matrix(df.chr)
eec = df.chr
save(sites, samps, afmat, eec, file = './orch2021_Downsampled_META_Filtered.RData')

In [20]:
#Now get just E Cage RData
dim(samps)
df = cbind(samps, t(afmat))
df.eec = cbind(samps, t(eec))

df = df %>% filter(treatment == 'E' & tech.rep == 'No' & biol.rep == 'No')
df.eec = df.eec %>% filter(treatment == 'E' & tech.rep == 'No' & biol.rep == 'No')

samps = df[,1:ncol(samps)]
afmat = df[,-c(1:ncol(samps))]
afmat = t(afmat)

eec = df.eec[,-c(1:ncol(samps))]
eec = t(eec)

save(sites, samps, afmat, eec, file = './orch2021_Downsampled_ECage_Filtered.RData')

[1] 318   7

In [23]:
afmat = df[,-c(1:ncol(samps))]
afmat = t(afmat)

eec = df.eec[,-c(1:ncol(samps))]
eec = t(eec)
save(sites, samps, afmat, eec, file = './orch2021_Downsampled_ECage_Filtered.RData')

In [29]:
##Doing same for baseline
load('./orch2021_Baseline_Downsampled_Filtered.RData', verbose = TRUE)
n.2L = nrow(sites %>% filter(chrom == '2L'))
n.2R = nrow(sites %>% filter(chrom == '2R'))
n.3L = nrow(sites %>% filter(chrom == '3L'))
n.3R = nrow(sites %>% filter(chrom == '3R'))
n.X = nrow(sites %>% filter(chrom == 'X'))

Loading objects:
  sites.base
  samps.base
  afmat.base
  eec.base


In [ ]:
##Doing same for baseline
load('./orch2021_Baseline_Downsampled_Filtered.RData')
n.2L = nrow(sites.base %>% filter(chrom == '2L'))
n.2R = nrow(sites.base %>% filter(chrom == '2R'))
n.3L = nrow(sites.base %>% filter(chrom == '3L'))
n.3R = nrow(sites.base %>% filter(chrom == '3R'))
n.X = nrow(sites.base %>% filter(chrom == 'X'))
df = cbind(sites.base, eec.base)
df.chr = df %>% group_by(chrom) %>% sample_n(1)
df.chr = df.chr[,-c(1:2)]
df.chr = sapply(df.chr,"*",(1/3.33))
df.chr = as.data.frame(df.chr)
df.chr = df.chr %>% slice(rep(1:n(), times = c(n.2L, n.2R, n.3L, n.3R, n.X)))
df.chr = as.matrix(df.chr)
eec.base = df.chr
save(eec.base, samps.base, afmat.base, eec.base, file = './orch2021_Baseline_Downsampled_Filtered.RData')

In [37]:
#doing same for baseline mean RDaata
load('./orch2021_Baseline_Downsampled_Filtered_Mean.RData')
n.2L = nrow(sites.base %>% filter(chrom == '2L'))
n.2R = nrow(sites.base %>% filter(chrom == '2R'))
n.3L = nrow(sites.base %>% filter(chrom == '3L'))
n.3R = nrow(sites.base %>% filter(chrom == '3R'))
n.X = nrow(sites.base %>% filter(chrom == 'X'))
df = cbind(sites.base, eec.base.mean)
df.chr = df %>% group_by(chrom) %>% sample_n(1)
df.chr = df.chr[,-c(1:2)]
df.chr = sapply(df.chr,"*",(1/3.33))
df.chr = as.data.frame(df.chr)
df.chr = df.chr %>% slice(rep(1:n(), times = c(n.2L, n.2R, n.3L, n.3R, n.X)))
df.chr = as.matrix(df.chr)
eec.base.mean = df.chr

save(eec.base.mean, afmat.base.mean, sites.base, file = 'orch2021_Baseline_Downsampled_Filtered_Mean.RData')

In [2]:
load('./orch2021_Downsampled_TechReps.RData')

In [3]:
df = cbind(samps, t(afmat))
df.eec = cbind(samps, t(eec))

In [4]:
df = df %>% filter(treatment == 'E')
df.eec = df.eec %>% filter(treatment == 'E')

In [5]:
samps = df[,1:ncol(samps)]

In [6]:
afmat = df[ , -c(1:ncol(samps))]
afmat = t(afmat)
eec = df.eec[,-c(1:ncol(samps))]
eec = t(eec)

In [7]:
save(afmat, eec, sites, samps, file = './orch2021_Downsampled_TechReps_ECages.RData')